In [1]:
# 기본 라이브러리 임포트 
import os
import pandas as pd
import numpy as np
from IPython.display import display
import time

# 시각화 라이브러리 임포트
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# 피클 관련 라이브러리 임포트
import pickle
import gzip

# 기본 sklearn 모듈 임포트
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, f1_score

# imbalanced data processing 관련 라이브러리 임포트
from imblearn.combine import SMOTEENN, SMOTETomek
from collections import Counter

# XGBoost 관련 모듈 임포트
import xgboost as xgb
from xgboost import plot_importance

%matplotlib inline

### shap으로 추출한 변수들만 포함한 user 데이터

In [ ]:
with gzip.open('../preprocessed/user_final.pickle','rb') as f:
    user = pickle.load(f)

user.head()

In [4]:
user.shape

(723526, 326)

In [ ]:
# 모든 열 이름 표시
pd.set_option('display.max_seq_items', None)
user.columns

In [6]:
# final features for user data (shap으로 추출한 변수)
user_shap = ['income_type', 'employment_type', 'houseown_type', 'purpose',
            'credit_score', 'desired_amount', 'purpose_by_income_type_SWITCHLOAN', 
            'insert_hour', 'existing_loan_amt', 'yearly_income', 'existing_loan_amt_by_credit_score_cut_quantile_first', 
            'company_period', 'existing_loan_cnt', 'income_type', 'desired_amount_by_income_type_std', 
            'existing_loan_amt_by_purpose_max', 'insert_day', 'gender', 'age', 'insert_month', 
            'yearly_income_by_houseown_type_mean', 'purpose_by_income_type_BUYCAR', 'employment_type', 'insert_minute']

len(user_shap)

24

In [ ]:
# user 데이터프레임에 shap으로 뽑은 feature, is_applied, 그리고 인덱스로 사용할 열 가져와서 user에 붙이기
user = user[user_shap + ['is_applied', 'user_id', 'application_id']]
user.head()

In [ ]:
user.shape

### log data 

In [ ]:
# 군집화 위해 user 테이블과 합칠 log 데이터 불러오기
with gzip.open('../preprocessed/log_final.pickle','rb') as f:
    log = pickle.load(f)

log.head()

In [ ]:
log.columns

In [ ]:
len(log.columns)

In [ ]:
log.shape

### merge (user + log)

In [14]:
# user 데이터프레임의 인덱스를 'user_id'로 설정
user.set_index(['user_id'], inplace=True)

In [15]:
user.shape

(723526, 26)

In [ ]:
# user 데이터 형태
user.head()

In [ ]:
# log 데이터 형태
log.head()

In [ ]:
# user과 log 합치기
merge_user_log = pd.merge(left=user, right=log, left_index=True, right_index=True, how="inner")
merge_user_log.head()

In [28]:
# 시간 형태 바꿔주기
temp_time = merge_user_log['insert_month'].astype(str)+merge_user_log['insert_day'].astype(str)+merge_user_log['insert_hour'].astype(str)+merge_user_log['insert_minute'].astype(str)
temp_time = temp_time.astype('int')

In [ ]:
# 바꾼 결과
temp_time

In [29]:
# 위에서 새로 만든 값들을 merge한 테이블의 'insert_time'으로 추가
merge_user_log['insert_time']=temp_time

In [31]:
# sort and drop duplicates
aa = merge_user_log.reset_index(drop=False).sort_values(['user_id','insert_time']).drop_duplicates(subset=['user_id'], keep='last')

In [33]:
# set index and drop column
bb = aa.set_index('user_id').drop('insert_time',axis=1)

In [35]:
bb.shape

(183239, 50)

In [36]:
# save
with gzip.open('../preprocessed/forcluster_final.pickle','wb') as f:
    pickle.dump(bb, f, pickle.HIGHEST_PROTOCOL)